# SITCOM-1120: Brake analysis

# Data
#### e-stop - rolex
 * 22-11-2023
 0.  Fisrt log TMA:  2023-11-22T19:56:33.622523   /// bquint@love01.cp.lsst.org  /
 1.  by 2023-11-23T00:51:28 / running the TMA Az/El breaks tests( E-stop ) /// ashugart@love01.cp.lsst.org 
 2.  by 2023-11-23T02:32:17.254532 /    bquint@love01.cp.lsst.org 
   * We ran another TMA Brake Test earlier today. Taking note now. 2023-11-23 00:38 UTC - 2023-11-23 00:42 UTC
   * TMA Brake tests in Azimuth - Going from 151º to 0º
   * We stopped around 102.5º, and we overshot until 89.67º
 3. 2023-11-23T03:08:50.50 /   bquint@love01.cp.lsst.org 
   * - 2023-11-23 01:02 UTC - 2023-11-23 01:05 UTC
   * - TMA Brake tests in Elevation - Going up from 20º to 80º.
   * - We stopped around 40º, and we overshooted until 45º.
 4.  2023-11-23 01:35 UTC - 2023-11-23 01:40 UTC / by bquint@love01.cp.lsst.org 2023-11-23T03:10:09
   * - TMA Brake tests in Elevation - Going down from 80º to 20º.
   * - We stopped around 57.8º, and we overshooted until 53.8º.
    
---

 * BLOCK-142 : https://jira.lsstcorp.org/browse/BLOCK-142




In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
from datetime import datetime
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from scipy.interpolate import UnivariateSpline

client = EfdClient("usdf_efd")

%load_ext lab_black
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Functions

In [ ]:
def format_time(t_start, t_end):
    start = Time(t_start, scale="utc", format="isot")
    end = Time(t_end, scale="utc", format="isot")
    return (start, end)


def plot_az(az, azXs, azVs, event=None):
    plt.figure(figsize=(16, 15))
    plt.suptitle("lsst.sal.MTMount.azimuth", fontsize=18, fontweight="bold")

    # Position
    plt.subplot(4, 1, 1)
    plt.plot(
        az["actualPosition"],
        label="Position",
    )
    plt.plot(
        az["demandPosition"],
        label="demandPosition",
    )
    # If df_events is provided, plot vertical lines
    if event is not None:
        plt.axvline(x=event, color="r", linestyle="--")

    plt.xlabel("Time", fontsize=14)
    plt.ylabel("Position", fontsize=14)
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend(fontsize=14)
    plt.xticks(fontsize=12.5, fontweight="bold")
    plt.yticks(fontsize=14)

    # Velocity
    plt.subplot(4, 1, 2)
    plt.plot(
        az["actualVelocity"],
        label="Velocity",
    )
    plt.plot(
        az["demandVelocity"],
        label="demandVelocity",
    )
    # If df_events is provided, plot vertical lines
    if event is not None:
        plt.axvline(x=event, color="r", linestyle="--")

    plt.xlabel("Time", fontsize=14)
    plt.ylabel("Velocity", fontsize=14)
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend(fontsize=14)
    plt.xticks(fontsize=12.5, fontweight="bold")
    plt.yticks(fontsize=14)

    # Acceleartion
    plt.subplot(4, 1, 3)
    plt.plot(
        elXs,
        elVs,
        label="Velocity",
    )
    plt.xlabel("Time", fontsize=14)
    plt.ylabel("Velocity", fontsize=14)
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend(fontsize=14)
    plt.xticks(fontsize=12.5, fontweight="bold")
    plt.yticks(fontsize=14)

    plt.subplot(4, 1, 4)
    plt.plot(
        elXs,
        elAccSpline(elXs),
        label="Interp. - Acceleration ",
    )
    plt.xlabel("Time", fontsize=14)
    plt.ylabel("Acceleration", fontsize=14)
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend(fontsize=14)
    plt.xticks(fontsize=12.5, fontweight="bold")
    plt.yticks(fontsize=14)

    # disign
    plt.tight_layout()
    plt.show()
    return


def plot_el(el, elXs, elVs, event=None):
    plt.figure(figsize=(16, 15))
    plt.suptitle("lsst.sal.MTMount.elevation", fontsize=18, fontweight="bold")

    # Position
    plt.subplot(4, 1, 1)
    plt.plot(
        el["actualPosition"],
        label="Position",
    )
    plt.plot(
        el["demandPosition"],
        label="demandPosition",
    )
    # If df_events is provided, plot vertical lines
    if event is not None:
        plt.axvline(x=event, color="r", linestyle="--")
    plt.xlabel("Time", fontsize=14)
    plt.ylabel("Position", fontsize=14)
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend(fontsize=14)
    plt.xticks(fontsize=12.5, fontweight="bold")
    plt.yticks(fontsize=14)

    # Velocity
    plt.subplot(4, 1, 2)
    plt.plot(
        el["actualVelocity"],
        label="Velocity",
    )
    plt.plot(
        el["demandVelocity"],
        label="demandVelocity",
    )
    # If df_events is provided, plot vertical lines
    if event is not None:
        plt.axvline(x=event, color="r", linestyle="--")
    plt.xlabel("Time", fontsize=14)
    plt.ylabel("Velocity", fontsize=14)
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend(fontsize=14)
    plt.xticks(fontsize=12.5, fontweight="bold")
    plt.yticks(fontsize=14)

    # Acceleartion
    plt.subplot(4, 1, 3)
    plt.plot(
        elXs,
        elVs,
        label="Velocity",
    )
    plt.xlabel("Time", fontsize=14)
    plt.ylabel("Velocity", fontsize=14)
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend(fontsize=14)
    plt.xticks(fontsize=12.5, fontweight="bold")
    plt.yticks(fontsize=14)

    plt.subplot(4, 1, 4)
    plt.plot(
        elXs,
        elAccSpline(elXs),
        label="Interpolated Acceleration ",
    )
    plt.xlabel("Time", fontsize=14)
    plt.ylabel("Acceleration", fontsize=14)
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend(fontsize=14)
    plt.xticks(fontsize=12.5, fontweight="bold")
    plt.yticks(fontsize=14)

    # If df_events is provided, plot vertical lines
    if event is not None:
        plt.axvline(x=event, color="r", linestyle="--")

    # disign
    plt.tight_layout()
    plt.show()
    return


def call_az_data_range(start, end):
    df_az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=[
            "actualPosition",
            "actualVelocity",
            "demandPosition",
            "demandVelocity",
            "timestamp",
        ],
        begin=start,
        end=end,
    )
    return df_az


def call_el_data_range(start, end):
    df_el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=[
            "actualPosition",
            "actualVelocity",
            "demandPosition",
            "demandVelocity",
            "timestamp",
        ],
        begin=start,
        end=end,
    )
    return df_el


def acc_spline(df_test):
    elXs = df_test["timestamp"].values - df_test["timestamp"].values[0]
    elVs = df_test["actualVelocity"].values
    elVelSpline = UnivariateSpline(elXs, elVs, s=0)
    elAccSpline = elVelSpline.derivative(n=1)
    return (elAccSpline, elXs, elVs)


def find_state(dataframe, column, number):
    """
    This function searches for a specific number in a specified column of a DataFrame
    and returns the rows that contain that number.

    Parameters:
    dataframe (pd.DataFrame): The DataFrame to search.
    column (str): The column name to search within.
    number (int or float): The number to search for.

    Returns:
    pd.DataFrame: A DataFrame containing the rows where the specified column contains the number.
    """
    # Filter the DataFrame to find the rows where the specified column contains the number
    rows_with_number = dataframe[dataframe[column] == number]

    return rows_with_number


def get_event(start, end):
    """
    Fetch an event from a Axis stopped in a defined range of time.
    This need the find_state() function in order to achieve the goal.

    Parameters:
     start, end : range of time to search.
    """
    df_az = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_azimuthMotionState",
        columns=[
            "state",
            "timestamp",
        ],
        begin=start,
        end=end,
    )

    df_el = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_elevationMotionState",
        columns=[
            "state",
            "timestamp",
        ],
        begin=start,
        end=end,
    )

    # Searches for value=1 in state column
    colum = "state"
    value = 1

    result_az = find_state(df_az, colum, value)
    result_el = find_state(df_el, colum, value)

    # Check if the result is empty
    if result_az.empty and (not result_el.empty):
        print("The Elevation has stopped.")
        result = result_el
        # Reiniciar el índice
        result.reset_index(inplace=True)
        # Asignar un nombre al índice
        result.index.name = "Elevation"
        result = result.rename(columns={"index": "time"})
        print(result)
    elif result_el.empty and (not result_az.empty):
        print("The Azimuth has stopped.")
        result = result_az
        result.reset_index(inplace=True)
        # Asignar un nombre al índice
        result.index.name = "Azimuth"
        result = result.rename(columns={"index": "time"})
        print(result)
    elif (not result_el.empty) and (not result_az.empty):
        print("Both axis found in Stopped State")
        print("This is a case that needs a deep look")
        print("Elevation")
        print(result_el.empty)
        print("Azimuth")
        print(result_az.empty)
        # MAYBE concadenated both dataframe in this section?
        result = 0
        result_el.index.name = "Elevation"
        print(result_el)
        result_az.index.name = "Azimuth"
        print(result_az)
    else:
        print("Data is not retrieving information about axis stopped")
        result = 0
        print(result)

    return result

### df_error : 
* **active**: IDL_Type: boolean / Units: unitless / Description: Is the error condition present?
* **text**: IDL_Type: string / Units: unitless / Description: Text description of err
* **subsystem**: IDL_Type:long / Units: unitless
* **Description**: Subsystem that produced the error; a single System enumeration value.

* **MTMount_logevent_elevationMotionState**: elevationMotionState ?? another keyword /  Description: Motion state of the elevation axis. / EFDB_Topic: MTMount_logevent_elevationMotionState


### AxisMotionState:
* Stopping=0

* Stopped=1

* MovingPointToPoint=2

* Jogging=3

* Tracking=4

* TrackingPaused=5

### Causes and Effects
* E-Stop - Any other - Control Room - gcauses_D4

IDL_Type:
boolean

* E-Stop - Dome ETPB’s. - gcauses_D12

IDL_Type:
boolean

* E-Stop  TMA ETPB’s - gcauses_D9

IDL_Type:
boolean

* Description: TMA main drives STO and engage the brakes. - geffects_A5

## Check of GIS matrix
* SITCOM-1391: Software Component - GIS


# Running 

## Fesching data. # Main function:
 1.  Observing time started at : Fisrt log TMA:  2023-11-22T19:56:33.622523   /// bquint@love01.cp.lsst.org  /

In [ ]:
start, end = format_time("2023-11-22T20:05:00", "2023-11-23T06:07:00")

# df_el = call_el_data_range(start, end)
# elAccSpline, elXs, elVs = acc_spline(df_el)
# plot_el(df_el, elXs, elVs, df_events=None)

# Case 1:
 1.  by 2023-11-23T00:51:28 / running the TMA Az/El breaks tests( E-stop ) /// ashugart@love01.cp.lsst.org 


In [ ]:
##################################################
time_start = "2023-11-23T00:39:00"
time_ends = "2023-11-23T00:55:00"
##################################################
start, end = format_time(time_start, time_ends)
df_time_stop = get_event(start, end)
##################################################

In [ ]:
##################################################
# df_el = call_el_data_range(start, end)
# elAccSpline, elXs, elVs = acc_spline(df_el)
# plot_el(df_el, elXs, elVs, event=None)
##################################################

# Case 2:
 2.  by 2023-11-23T02:32:17.254532 /    bquint@love01.cp.lsst.org 
   * We ran another TMA Brake Test earlier today. Taking note now. 2023-11-23 00:38 UTC - 2023-11-23 00:42 UTC
   * TMA Brake tests in Azimuth - Going from 151º to 0º
   * We stopped around 102.5º, and we overshot until 89.67º


In [ ]:
##################################################
time_start = "2023-11-23T00:40:00"
time_ends = "2023-11-23T00:47:00"
##################################################
start, end = format_time(time_start, time_ends)
df_time_stop = get_event(start, end)
##################################################

In [ ]:
##################################################
time_stop = str(df_time_stop["time"][0])
event = datetime.strptime(time_stop, "%Y-%m-%d %H:%M:%S.%f%z").strftime("%H:%M:%S.%f")
##################################################
df_el = call_el_data_range(start, end)
elAccSpline, elXs, elVs = acc_spline(df_el)
#plot_el(df_el, elXs, elVs, event)
##################################################

# Case 3:
 3. 2023-11-23T03:08:50.50 /   bquint@love01.cp.lsst.org 
   * - 2023-11-23 01:02 UTC - 2023-11-23 01:05 UTC
   * - TMA Brake tests in Elevation - Going up from 20º to 80º.
   * - We stopped around 40º, and we overshooted until 45º.


In [ ]:
time_start = "2023-11-23T01:03:50"
time_ends = "2023-11-23T01:04:15"

# Case 4:
 4.  2023-11-23 01:35 UTC - 2023-11-23 01:40 UTC / by bquint@love01.cp.lsst.org 2023-11-23T03:10:09
   * - TMA Brake tests in Elevation - Going down from 80º to 20º.
   * - We stopped around 57.8º, and we overshooted until 53.8º.

In [ ]:
time_start = "2023-11-23T01:43:00"
time_ends = "2023-11-23T02:00:00"

## Next:
1. ~~Summary state error , search for logs .. each one~~
2. Calculate distance after e-stop
3. ~~Search in GIS. / Search for FAULT~~
4. ~~Search for cmd_fault an E-Stop words?~~
5. Determind by diff ways the distance, -
6. ~~Do it automatics searsh of e-stops~~